<a href="https://colab.research.google.com/github/CatTaborEP/ProjectMLUTEP/blob/main/MLProject_Take3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
import torch
from torch import nn
!pip install datasets
!pip install pytorch-transformers
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
import accelerate
import transformers


transformers.__version__, accelerate.__version__





('4.38.2', '0.28.0')

# Apply BERT?????Universal Sentence Encoder???
multilabel vs multiclass
Multi-label would allow for classification based on both text level and appropriateness
Multi-class would allow for multiple classifications rather than just 2.


In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
BASE_MODEL = "bert-base-uncased"
LEARNING_RATE = 2e-5
MAX_LENGTH = 512
BATCH_SIZE = 16
EPOCHS = 20

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_df = pd.read_csv('lexile.csv')
print(len(train_df))


4724


In [ ]:

model.classifier = nn.Linear(768, 1)
model.num_labels = 1

In [ ]:

train_datasets = load_dataset('csv', data_files='lexile.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['Excerpt'], padding='max_length', truncation=True, max_length=512)

In [ ]:
f_train_datasets = train_datasets.map(tokenize_function, batched=True)


Map:   0%|          | 0/4724 [00:00<?, ? examples/s]

In [ ]:
f_train_datasets = f_train_datasets.remove_columns(['ID', 'Last Changed', 'Author', 'Title', 'Anthology', 'URL', 'Source',
       'Pub Year', 'Category', 'Location', 'License', 'MPAA\nMax',
       'MPAA \n#Max', 'MPAA\n#Avg', 'Excerpt', 'Google\nWC', 'Joon\nWC v1',
       'British WC', 'British Words', 'Sentence\nCount v1',
       'Sentence\nCount v2', 'Paragraphs', 'BT s.e.',
       'Flesch-Reading-Ease', 'Flesch-Kincaid-Grade-Level',
       'Automated Readability Index', 'SMOG Readability',
       'New Dale-Chall Readability Formula', 'CAREC', 'CAREC_M', 'CARES',
       'CML2RI', 'firstPlace_pred', 'secondPlace_pred', 'thirdPlace_pred',
       'fourthPlace_pred', 'fifthPlace_pred', 'sixthPlace_pred',
       'Kaggle split'])


In [ ]:
f_train_datasets = f_train_datasets.rename_column('BT Easiness', 'labels')


In [ ]:
print(f_train_datasets)

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4724
    })
})


In [ ]:
n_samples = len(f_train_datasets['train'])
n_train = int(0.4 * n_samples)
print(n_train)

1889


In [ ]:
x_train = f_train_datasets['train'].select(range(n_train))
x_test = f_train_datasets['train'].select(range(n_train, n_samples))
n_samples = len(x_train)
print(n_samples)
n_train = int(0.9 * n_samples)
x_eval = x_train.select(range(n_train, n_samples))
x_train = x_train.select(range(n_train))
print(n_train)

print(x_train)
print(x_eval)


1889
1700
Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1700
})
Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 189
})


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits, labels = logits.squeeze(), labels.squeeze()
    rmse = np.sqrt(np.mean((labels - logits) ** 2))
    return {'RMSE': rmse}

In [17]:
training_args = TrainingArguments(
    'training_args',
    num_train_epochs = 5,
    logging_steps = 200,
    learning_rate = 1e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    evaluation_strategy = 'steps'
)

trainer = Trainer(
    model = model,
    train_dataset = x_train,
    eval_dataset = x_eval,
    compute_metrics = compute_metrics,
    args = training_args
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`